In [1]:
%%writefile test-simple.md
~~~csv
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-simple.md


In [2]:
%%writefile test-full.md
~~~csv
title: "*Great* Title"
has-header: False
column-width: 0.25, 0.25, 0.25, 0.25
table-width: 1
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-full.md


In [3]:
#!/usr/bin/env python3

"""
Panflute filter to parse CSV in fenced YAML code blocks
e.g.

```markdown
~~~csv
title: "*Great* Title"
has-header: False
column-width: 0.25, 0.25, 0.25, 0.25
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~
```
"""

import io
import csv
import panflute

def fenced_csv(options, data, element, doc):
    # read csv and convert to panflute table representation
    with io.StringIO(data) as f:
        reader = csv.reader(f)
        body = []
        for row in reader:
            cells = [panflute.TableCell(*panflute.convert_text(x)) for x in row]
            body.append(panflute.TableRow(*cells))
        # get no of columns for header
        f.seek(0)
        noOfColumn = len(list(reader)[0])
    # read YAML metadata
    try:
        caption = options.get('title')
        column_width = options.get('column-width')
        alignment = options.get('alignment')
        has_header = options.get('has-header',True)
    except AttributeError:
        caption = None
        column_width = None
        alignment = None
        has_header = True
    # get caption
    if caption != None:
        caption = panflute.convert_text(caption)[0].content
    # get column_width
    if column_width != None:
        column_width = [float(x) for x in column_width.split(",")]
    # get alignment
    if alignment != None:
        alignment = [x.strip() for x in alignment.split(",")]
    # finalize table according to metadata
    header = body.pop(0) if has_header else panflute.TableRow(*[panflute.TableCell() for i in range(noOfColumn)])
    table = panflute.Table(*body, header=header, caption=caption, width=column_width, alignment=alignment)
    return table

# We'll only run this for CodeBlock elements of class 'csv'
if __name__ == '__main__':
    panflute.toJSONFilter(panflute.yaml_filter, tag='csv', function=fenced_csv)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
!chmod +x ../bin/csv-tables.py
print("JSON", "Simple", "="*80)
!pandoc test-simple.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t native
print("\n", "Markdown", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t markdown
print("JSON", "Full", "="*80)
!pandoc test-full.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t native
print("\n", "Markdown Grid", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t markdown
print("\n", "Markdown Multiline", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t markdown-grid_tables